### Aufgaben

Hier werden für den Stichprobenumfang n=100

- zur Wahrscheinlichkeit p=0,26 das 95%-Prognoseintervall,
- zur relativen Häufigkeit h=0,18 das 95%-Konfidenzintervall
- exakt und auch näherungsweise berechnet.

a) Zeigen Sie, dass die Ergebnisse stimmen.

b) Fassen Sie die Bedeutung der gelben und der grünen Strecke in Worte.

c) Elif: "Ich kann der Abbildung die Antwort auf die Frage entnehmen, ob die relative Häufigkeit h=0,18 Anlass bietet, die Wahrscheinlichkeit p=0,26 zu bezweifeln." Erläutern Sie Elifs Aussage und zu welchem Ergebnis Elif kommt.

d) Ümü: "Die Antwort auf Elifs Frage hängt davon ab, ob man exakt oder mit der Näherung arbeitet." Prüfen und kommentieren Sie.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy.stats import norm, beta
import warnings

warnings.simplefilter("ignore")

def wilson_ci_bounds(h,n,c):
    a=1/(1+c**2/n)
    term= a*c*np.sqrt(1/(4*n)*(1-a)/a+h*(1-h)/n)
    lower_bound = (1-a)*0.5+a*h-term
    upper_bound = (1-a)*0.5+a*h+term
    return lower_bound, upper_bound

def clopper_pearson_ci(h, n,c):
    alpha = 2 * (1 - norm.cdf(c))  # Berechnung von alpha aus c
    lower_bound = beta.ppf(alpha / 2, h, n - h + 1)
    upper_bound = beta.ppf(1 - alpha / 2, h + 1, n - h)
    return lower_bound, upper_bound

def wald_ci(h, n, c):
    se = np.sqrt(h * (1 - h) / n)
    lower_bound = h - c * se
    upper_bound = h + c * se
    return max(0, lower_bound), min(1, upper_bound)

def plot_confidence_ellipse(h, n, c, p_value):
    p = np.linspace(0, 1, 1000)  # p-Werte als x-Achse
    f1 = p - c * np.sqrt(p * (1 - p) / n)
    f2 = p + c * np.sqrt(p * (1 - p) / n)
    lower_wilson, upper_wilson = wilson_ci_bounds(h, n, c)
    lower_cp, upper_cp = clopper_pearson_ci(h * n, n,c)
    lower_wald, upper_wald = wald_ci(h, n, c)
    
    # Prognoseintervall für gegebenes p
    f1_p = p_value - c * np.sqrt(p_value * (1 - p_value) / n)
    f2_p = p_value + c * np.sqrt(p_value * (1 - p_value) / n)
    
    plt.figure(figsize=(6,6))
    plt.plot(p, f1, 'r', label="$f_-(p)$")
    plt.plot(p, f2, 'b', label="$f_+(p)$")
    plt.axhline(h, color='gray', linestyle='dashed')
    plt.axvline(p_value, color='gray', linestyle='dashed')
    
    # Prognoseintervall
    plt.plot([p_value, p_value], [f1_p, f2_p], 'm-', linewidth=3, label="Prognoseintervall")
    
    # Wilson-Konfidenzintervall (grün)
    plt.plot([lower_wilson, upper_wilson], [h, h], 'g-', linewidth=3, label="Wilson-KI")
    
    plt.scatter([lower_wilson, upper_wilson], [h, h], color='black', s=20, zorder=3)
    plt.scatter([p_value, p_value], [f1_p, f2_p], color='black', s=20, zorder=3)
    plt.fill_between(p, f1, f2, color='gray', alpha=0.1)
    
    # Punkte (0,h) und (p,0)
    plt.scatter([0, p_value], [h, 0], color='black', marker='o', zorder=3)
    
    plt.xlabel("p (unbekannter Parameter)",fontsize=14)
    plt.ylabel("h (Stichprobenergebnis)",fontsize=14)
    plt.legend(fontsize=11)
    plt.title(f"Wilson-Konfidenzellipse ({round(norm.cdf(c) * 2 - 1, 3) * 100}% Konfidenzintervall)",
              fontsize=14)
    plt.grid(True)
    plt.show()
    
    print(f"Prognoseintervall für p={p_value:.2f}: [{f1_p:.4f}, {f2_p:.4f}]")
    print()
    print(f"Wilson-Konfidenzintervall: [{lower_wilson:.4f}, {upper_wilson:.4f}]")
    print(f"Clopper-Pearson-Konfidenzintervall: [{lower_cp:.4f}, {upper_cp:.4f}]")
    print(f"Wald-Konfidenzintervall: [{lower_wald:.4f}, {upper_wald:.4f}]")
   

def interactive_plot(c, n, h, p_value):
    plot_confidence_ellipse(h, n, c, p_value)

c_slider = widgets.FloatSlider(min=1.0, max=3.0, step=0.01, value=1.96, description="c")
n_slider = widgets.IntSlider(min=10, max=1000, step=10, value=100, description="n")
h_slider = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, value=0.18, description="h")
p_slider = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, value=0.26, description="p")

widgets.interactive(interactive_plot, c=c_slider, n=n_slider, h=h_slider, p_value=p_slider)

interactive(children=(FloatSlider(value=1.96, description='c', max=3.0, min=1.0, step=0.01), IntSlider(value=1…

# Konfidenzintervalle und deren Visualisierung

Dieses Python-Programm bietet eine interaktive Visualisierung von Konfidenzintervallen für einen unbekannten Parameter \( p \), basierend auf verschiedenen statistischen Methoden. Es zeigt, wie sich die Breite der Konfidenzintervalle verändert, je nach den Eingabewerten für Stichprobengröße, Erfolgsrate und Konfidenzlevel.

---
## 1. Einführung zu Konfidenzintervallen

Ein **Konfidenzintervall** ist ein Bereich von Werten, der den wahren Wert eines Parameters mit einer bestimmten Wahrscheinlichkeit enthält. In diesem Fall versuchen wir, das Konfidenzintervall für einen Parameter \( p \) (z. B. den Anteilswert einer bestimmten Eigenschaft in einer Population) zu berechnen, basierend auf einer Stichprobe. Die Breite des Intervalls hängt von mehreren Faktoren ab, einschließlich der Stichprobengröße und des Konfidenzlevels.

Wir betrachten in diesem Programm drei verschiedene Methoden zur Berechnung von Konfidenzintervallen:
- **Wilson-Konfidenzintervall**
- **Clopper-Pearson-Konfidenzintervall**
- **Wald-Konfidenzintervall**

Jede dieser Methoden hat ihre eigenen Eigenschaften und Vorteile, abhängig von der Situation und der Größe der Stichprobe.

---
## 2. Das Programm

### Bibliotheken

Das Programm verwendet die folgenden Bibliotheken:
- **NumPy**: Zum Berechnen von mathematischen Funktionen.
- **Matplotlib**: Zum Erstellen von Grafiken.
- **ipywidgets**: Für interaktive Widgets.
- **SciPy**: Für statistische Berechnungen (z. B. den Clopper-Pearson-Test).
- **import warnings**: Warnmeldung bei Clopper-Pearson verhindern

---
### Funktionen im Programm

#### 2.1 Berechnung der Wilson-Konfidenzintervall-Grenzen

Die Funktion `wilson_ci_bounds` berechnet die unteren und oberen Grenzen des **Wilson-Konfidenzintervalls**. Diese Methode ist besonders nützlich, da sie bei kleinen Stichprobengrößen eine genauere Schätzung liefert als das einfache Wald-Konfidenzintervall.

```python
def wilson_ci_bounds(h, n, c):
    a = 1 / (1 + c**2 / n)
    term = a * c * np.sqrt(1 / (4 * n) * (1 - a) / a + h * (1 - h) / n)
    lower_bound = (1 - a) * 0.5 + a * h - term
    upper_bound = (1 - a) * 0.5 + a * h + term
    return lower_bound, upper_bound
```
---
#### 2.2 Berechnung des Clopper-Pearson-Konfidenzintervalls
Die Funktion clopper_pearson_ci berechnet das Clopper-Pearson-Konfidenzintervall. Diese Methode basiert auf der inversen Beta-Verteilung und ist besonders für kleine Stichprobengrößen geeignet.


```python
def clopper_pearson_ci(h, n, c):
    alpha = 2 * (1 - norm.cdf(c))  # Berechnung von alpha aus c
    lower_bound = beta.ppf(alpha / 2, h, n - h + 1)
    upper_bound = beta.ppf(1 - alpha / 2, h + 1, n - h)
    return lower_bound, upper_bound
```
---
### 2.3 Berechnung des Wald-Konfidenzintervalls
Die Funktion wald_ci berechnet das Wald-Konfidenzintervall, welches das einfachste der drei Intervalle ist. Es kann jedoch bei kleinen Stichprobengrößen ungenau sein.


```python
def wald_ci(h, n, c):
    se = np.sqrt(h * (1 - h) / n)
    lower_bound = h - c * se
    upper_bound = h + c * se
    return max(0, lower_bound), min(1, upper_bound)
```
---
#### 2.4 Visualisierung der Konfidenzintervalle
Die Funktion plot_confidence_ellipse visualisiert die Konfidenzellipse. Hier werden die verschiedenen Konfidenzintervalle und das Prognoseintervall für einen gegebenen Wert von p
angezeigt. Die Konfidenzintervalle werden mit unterschiedlichen Farben dargestellt, und die Prognose für p wird als vertikale Linie hervorgehoben.

```python
def plot_confidence_ellipse(h, n, c, p_value):
    p = np.linspace(0, 1, 1000)  # p-Werte als x-Achse
    f1 = p - c * np.sqrt(p * (1 - p) / n)
    f2 = p + c * np.sqrt(p * (1 - p) / n)
    lower_wilson, upper_wilson = wilson_ci_bounds(h, n, c)
    lower_cp, upper_cp = clopper_pearson_ci(h * n, n, c)
    lower_wald, upper_wald = wald_ci(h, n, c)
    
    # Prognoseintervall für gegebenes p
    f1_p = p_value - c * np.sqrt(p_value * (1 - p_value) / n)
    f2_p = p_value + c * np.sqrt(p_value * (1 - p_value) / n)
    
    plt.figure(figsize=(6,6))
    plt.plot(p, f1, 'r', label="$f_-(p)$")
    plt.plot(p, f2, 'b', label="$f_+(p)$")
    plt.axhline(h, color='gray', linestyle='dashed')
    plt.axvline(p_value, color='gray', linestyle='dashed')
    
    # Prognoseintervall
    plt.plot([p_value, p_value], [f1_p, f2_p], 'm-', linewidth=3, label="Prognoseintervall")
    
    # Wilson-Konfidenzintervall (grün)
    plt.plot([lower_wilson, upper_wilson], [h, h], 'g-', linewidth=3, label="Wilson-KI")
    
    plt.scatter([lower_wilson, upper_wilson], [h, h], color='black', s=20, zorder=3)
    plt.scatter([p_value, p_value], [f1_p, f2_p], color='black', s=20, zorder=3)
    plt.fill_between(p, f1, f2, color='gray', alpha=0.1)
    
    # Punkte (0,h) und (p,0)
    plt.scatter([0, p_value], [h, 0], color='black', marker='o', zorder=3)
    
    plt.xlabel("p (unbekannter Parameter)", fontsize=14)
    plt.ylabel("h (Stichprobenergebnis)", fontsize=14)
    plt.legend(fontsize=11)
    plt.title(f"Wilson-Konfidenzellipse ({round(norm.cdf(c) * 2 - 1, 3) * 100}% Konfidenzintervall)",
              fontsize=14)
    plt.grid(True)
    plt.show()
    
    print(f"Prognoseintervall für p={p_value:.2f}: [{f1_p:.4f}, {f2_p:.4f}]")
    print()
    print(f"Wilson-Konfidenzintervall: [{lower_wilson:.4f}, {upper_wilson:.4f}]")
    print(f"Clopper-Pearson-Konfidenzintervall: [{lower_cp:.4f}, {upper_cp:.4f}]")
    print(f"Wald-Konfidenzintervall: [{lower_wald:.4f}, {upper_wald:.4f}]")

```
---
#### 2.5 Interaktive Widgets
Die Funktion interactive_plot nutzt die Schieberegler, um das Diagramm dynamisch zu ändern, basierend auf den Eingabewerten für c, n, h und p

```python
def interactive_plot(c, n, h, p_value):
    plot_confidence_ellipse(h, n, c, p_value)
```
---

#### 3. Widgets
Es gibt vier interaktive Schieberegler:
Konfidenzlevel c
c: Bestimmt das gewünschte Konfidenzlevel. Ein Wert von 1.96 entspricht z. B. einem 95%-Konfidenzintervall.
Stichprobengröße n
n: Die Anzahl der Beobachtungen in der Stichprobe.
Erfolgsrate h
h: Der Anteil der Erfolge in der Stichprobe.
Parameter p
p: Der unbekannte Parameter, dessen Konfidenzintervall du berechnen möchtest.

``` python
c_slider = widgets.FloatSlider(min=1.0, max=3.0, step=0.01, value=1.96, description="c")
n_slider = widgets.IntSlider(min=10, max=1000, step=10, value=100, description="n")
h_slider = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, value=0.18, description="h")
p_slider = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, value=0.26, description="p")
widgets.interactive(interactive_plot, c=c_slider, n=n_slider, h=h_slider, p_value=p_slider)

```
---
#### 4. Fazit
Dieses Programm bietet eine anschauliche Möglichkeit, Konfidenzintervalle für den Parameter 
p unter verschiedenen statistischen Methoden zu untersuchen. Es hilft dabei, zu verstehen, wie sich diese Intervalle je nach Stichprobengröße, Erfolgsrate und Konfidenzlevel ändern. Interaktive Widgets ermöglichen es, diese Auswirkungen in Echtzeit zu visualisieren.